In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse,signal

import matplotlib.pyplot as plt

import os,sys
sys.path.insert(0, '..')

from partial_trace import *
#from lanczos_bin import mystep,distribution

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload
%autoreload 2

/Users/muddy_flesh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
s = 1/2

M = int(2*s+1)

N = 12
n = M**N

N_s = 2
N_b = N-N_s
d_a = M**N_s
d_b = M**N_b

J_t = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        if np.abs(i-j)==1:
            J_t[i,j] = 1
            
J = 1

Jx = (J/2)*J_t
Jy = (J/2)*J_t
Jz = 0*J_t

In [3]:
# save system parameters
directory = f'data'
np.save(f'{directory}/dimensions.npy',[s,N,N_s],allow_pickle=True)
np.save(f'{directory}/couplings.npy',[Jx,Jy,Jz],allow_pickle=True)
H1 = get_hamiltonian(Jx,Jy,Jz,0,s)
H2 = get_hamiltonian(0*J_t,0*J_t,0*J_t,1,s)
sp.sparse.save_npz(f'{directory}/H1',H1)
sp.sparse.save_npz(f'{directory}/H2',H2)



In [4]:
# Find the jumping points at the ground states
h_min = 0
h_max = 2.5
iterations = 10
tolerance = 2e-2
turning_points = []
binary_search_recursive(lambda h: ground_state_function_smooth(h,H1,H2,d_a,d_b,k=5,β=1e5), h_min, h_max, iterations, tolerance, turning_points)
turning_points = np.hstack([[h_min],turning_points,[h_max]])

0,2.5,5.199e-01,2.220e-16,6.783e-01,10
0,1.25,5.199e-01,6.783e-01,5.825e-01,9
0,0.625,5.199e-01,5.825e-01,5.825e-01,8
0,0.3125,5.199e-01,5.825e-01,5.199e-01,7
0.15625,0.3125,5.199e-01,5.825e-01,5.199e-01,6
0.234375,0.3125,5.199e-01,5.825e-01,5.825e-01,5
0.234375,0.2734375,5.199e-01,5.825e-01,5.825e-01,4
0.234375,0.25390625,5.199e-01,5.825e-01,5.825e-01,3
0.234375,0.244140625,5.199e-01,5.825e-01,5.199e-01,2
0.2392578125,0.244140625,5.199e-01,5.825e-01,5.825e-01,1
0.625,1.25,5.825e-01,6.783e-01,6.895e-01,8
0.625,0.9375,5.825e-01,6.895e-01,6.895e-01,7
0.625,0.78125,5.825e-01,6.895e-01,5.825e-01,6
0.703125,0.78125,5.825e-01,6.895e-01,6.895e-01,5
0.703125,0.7421875,5.825e-01,6.895e-01,6.895e-01,4
0.703125,0.72265625,5.825e-01,6.895e-01,6.895e-01,3
0.703125,0.712890625,5.825e-01,6.895e-01,5.825e-01,2
0.7080078125,0.712890625,5.825e-01,6.895e-01,6.895e-01,1
1.25,2.5,6.783e-01,2.220e-16,1.744e-01,9
1.25,1.875,6.783e-01,1.744e-01,4.710e-01,8
1.25,1.5625,6.783e-01,4.710e-01,6.783e-01,7
1.40625,1

In [5]:
turning_points

array([0.        , 0.24047852, 0.70922852, 1.49780273, 1.77124023, 1.94213867, 2.5       ])

In [6]:
acc = (h_max-h_min)/2**iterations # accuracy of turning points
total_nodes = 100
num_nodes,hs = scaled_cheby(turning_points, total_nodes) # use chebyshev nodes to interpolate points between
hs = np.hstack([turning_points[1:]-acc,turning_points[:-1]+acc,hs]) # add points to the left and right of the boundaries

In [7]:
np.save(f'{directory}/turning_points.npy',turning_points)
np.save(f'{directory}/num_nodes.npy',num_nodes)

In [8]:
# Set the number of eigenvalues and deflations
k = 5
m = 25

In [9]:
# run the driver file to generate all the data files needed for the plot.
data_directory = f'data'
os.makedirs(f'{directory}/{data_directory}',exist_ok=True) # make subdirectory for data files

out_all = []
for hi,h in enumerate(hs):
    experiment_name = f'{h}'
    print(f'field strength: {hi} of {len(hs)}, {experiment_name}')
    clear_output(wait=True)
    os.system(f'python vN_entropy_phase_driver.py {h} {k} {m} {directory} {experiment_name} {data_directory}')

/Users/muddy_flesh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
